In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

import tensorflow as tf
from tensorflow import keras,lite

from keras.models import load_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [194]:
tf.version.VERSION

'2.10.0'

In [168]:
mp_hands = mp.solutions.hands # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results


def draw_styled_landmarks(image, results):
    # Draw left hand connections
    if not results.multi_hand_landmarks:
        return
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    
def draw_landmarks(image, results):
    if not results.multi_hand_landmarks:
        return
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw left hand connections
    
def extract_keypoints(results):
    lh = np.array([[res.landmark[point].x, res.landmark[point].y, res.landmark[point].z] for point in mp_hands.HandLandmark for res in results.multi_hand_landmarks], dtype=float).flatten() if results.multi_hand_landmarks else np.zeros(21*3*2)
    if lh.shape[0] == 63:
        rh = np.array([[res.landmark[point].x, res.landmark[point].y, res.landmark[point].z] for point in mp_hands.HandLandmark for res in results.multi_hand_landmarks], dtype=float).flatten() if results.multi_hand_landmarks else np.zeros(21*3)
        return np.concatenate([lh, rh])
    # rh = np.array([[res.landmark[point].x, res.landmark[point].y, res.landmark[point].z] for point in mp_hands.HandLandmark for res in results.multi_hand_landmarks], dtype=float).flatten() if results.multi_hand_landmarks else np.zeros(21*3*2)
    # return lh
    return np.concatenate([lh])


In [218]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'hii', 'by'])

# Thirty videos worth of data
no_sequences = 1

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [219]:
for action in actions: 
    for sequence in range(0,no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [220]:
try:
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        
        # NEW LOOP

        # Loop through actions
        for action in actions:
            # Loop through sequences aka videos
            for sequence in range(no_sequences):
                # Loop through video length aka sequence length
                for frame_num in range(sequence_length):

                    # Read feed
                    ret, frame = cap.read()

                    # Make detections
                    image, results = mediapipe_detection(frame, hands)

                    # Draw landmarks
                    draw_styled_landmarks(image, results)
                    
                    # NEW Apply wait logic
                    if frame_num == 0: 
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(500)
                    else: 
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)
                    
                    # NEW Export keypoints
                    keypoints = extract_keypoints(results)
                    # print(keypoints.shape)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)

                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                        
        cap.release()
        cv2.destroyAllWindows()

finally:
    # print("error")
    cap.release()
    cv2.destroyAllWindows()

In [87]:
cap.release()
cv2.destroyAllWindows()

In [221]:
label_map = {label: num for num, label in enumerate(actions)}
print(label_map)

{'hello': 0, 'hii': 1, 'by': 2}


In [222]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])


In [2]:
# print(sequences)
print(np.array(sequences).shape)
print(np.array(labels).shape)
X = np.array(sequences)
print(X.shape)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
print(y_test.shape)

NameError: name 'sequences' is not defined

In [247]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# model = Sequential()
# model.add(LSTM(64, return_sequences=True, input_shape=(30,126)))
# model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dense(64))
# model.add(Dense(32))
# model.add(Dense(actions.shape[0]))
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [245]:
# del model

In [248]:
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_50 (LSTM)              (None, 30, 64)            48896     
                                                                 
 lstm_51 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_52 (LSTM)              (None, 64)                49408     
                                                                 
 dense_56 (Dense)            (None, 64)                4160      
                                                                 
 dense_57 (Dense)            (None, 32)                2080      
                                                                 
 dense_58 (Dense)            (None, 3)                 99        
                                                                 
Total params: 203,459
Trainable params: 203,459
Non-t

In [226]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])


Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.0993 - categorical_accuracy: 0.9333
Epoch 2/100
1/1 [==============================] - 0s 421ms/step - loss: 1.0946 - categorical_accuracy: 0.9333
Epoch 3/100
1/1 [==============================] - 0s 434ms/step - loss: 1.0893 - categorical_accuracy: 0.9333
Epoch 4/100
1/1 [==============================] - 1s 516ms/step - loss: 1.0835 - categorical_accuracy: 0.9333
Epoch 5/100
1/1 [==============================] - 0s 419ms/step - loss: 1.0770 - categorical_accuracy: 0.9667
Epoch 6/100
1/1 [==============================] - 0s 397ms/step - loss: 1.0694 - categorical_accuracy: 0.9667
Epoch 7/100
1/1 [==============================] - 0s 433ms/step - loss: 1.0606 - categorical_accuracy: 0.9667
Epoch 8/100
1/1 [==============================] - 0s 444ms/step - loss: 1.0498 - categorical_accuracy: 0.9667
Epoch 9/100
1/1 [==============================] - 0s 412ms/step - loss: 1.0346 - categorical_accuracy: 0.9667
Epoc

In [227]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_35 (LSTM)              (None, 30, 64)            48896     
                                                                 
 lstm_36 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_37 (LSTM)              (None, 64)                49408     
                                                                 
 dense_41 (Dense)            (None, 64)                4160      
                                                                 
 dense_42 (Dense)            (None, 32)                2080      
                                                                 
 dense_43 (Dense)            (None, 3)                 99        
                                                                 
Total params: 203,459
Trainable params: 203,459
Non-t

In [228]:
res = model.predict(X_test)
res

1/1 [==============================] - 0s 477ms/step


array([[1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [232]:
position = 0
print(actions[np.argmax(res[position])])
print(actions[np.argmax(y_test[position])])

hello
hello


In [260]:
# model.save("action.h5")
# tf.keras.models.save_model(model,"saved_model.pb")
tf.keras.models.save_model(model,"action.h5")
# converter = lite.TFLiteConverter.from_keras_model(model)
# tfmodel = converter.convert()

# open(os.path.join(DATA_PATH, "action.tflite"),"wb").write(tfmodel)
# del model
model.load_weights("action.h5")

INFO:tensorflow:Assets written to: saved_model.pb\assets


INFO:tensorflow:Assets written to: saved_model.pb\assets


In [262]:
model.save(filepath="save_model/")
converter = tf.lite.TFLiteConverter.from_saved_model('save_model')
tflite_model = converter.convert()
open("action.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: save_model/assets


INFO:tensorflow:Assets written to: save_model/assets


ConverterError: <unknown>:0: error: loc(callsite(callsite(fused["TensorListReserve:", "sequential_16/lstm_35/TensorArrayV2_1@__inference__wrapped_model_347747"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_350140"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): 'tf.TensorListReserve' op requires element_shape to be static during TF Lite transformation pass
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: error: loc(callsite(callsite(fused["TensorListReserve:", "sequential_16/lstm_35/TensorArrayV2_1@__inference__wrapped_model_347747"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_350140"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): failed to legalize operation 'tf.TensorListReserve' that was explicitly marked illegal
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitio

In [ ]:
converter = lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()

open(os.path.join(DATA_PATH, "action.tflite"),"wb").write(tfmodel)

In [261]:
model = load_model('action.h5')
converter = lite.TFLiteConverter.from_saved_model("")
# converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("action_new.tflite", "wb").write(tflite_model)

UnknownError: NewRandomAccessFile failed to Create/Open: saved_model.pb : Access is denied.
; Input/output error

In [250]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 1s 556ms/step


array([[[0, 0],
        [0, 2]]], dtype=int64)

In [251]:
accuracy_score(ytrue, yhat)

1.0

In [89]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For static images:
IMAGE_FILES = []
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx, file in enumerate(IMAGE_FILES):
    # Read an image, flip it around y-axis for correct handedness output (see
    # above).
    image = cv2.flip(cv2.imread(file), 1)
    # Convert the BGR image to RGB before processing.
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print handedness and draw hand landmarks on the image.
    print('Handedness:', results.multi_handedness)
    if not results.multi_hand_landmarks:
      continue
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
    
cap.release()
cv2.destroyAllWindows()